<a href="https://colab.research.google.com/github/baronase/ml-residual-pendulum/blob/main/notebooks/pendulum_residual_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "gymnasium[classic-control]" stable-baselines3 torch numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 3.8 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import numpy as np

env = gym.make("Pendulum-v1")
obs, info = env.reset(seed=0)

print("obs:", obs)
print("obs shape:", obs.shape)
print("action space:", env.action_space)
print("obs space:", env.observation_space)

# take a few random steps
for i in range(5):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(i, "action:", action, "reward:", reward)


obs: [ 0.6520163   0.758205   -0.46042657]
obs shape: (3,)
action space: Box(-2.0, 2.0, (1,), float32)
obs space: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)
0 action: [-0.5652854] reward: -0.7620748568468063
1 action: [1.3246343] reward: -0.7443836493898092
2 action: [1.0806912] reward: -0.8761252845586246
3 action: [1.9851218] reward: -1.1987946893045602
4 action: [-1.0575279] reward: -1.82021578872846


In [3]:
import math

def obs_to_theta_theta_dot(obs):
    cos_t, sin_t, theta_dot = obs
    theta = math.atan2(sin_t, cos_t)  # in [-pi, pi]
    return theta, float(theta_dot)

theta, theta_dot = obs_to_theta_theta_dot(obs)
print("theta:", theta, "theta_dot:", theta_dot)


theta: 1.250038370404206 theta_dot: 2.971787452697754
